In [51]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [52]:
from torchvision.models.resnet import ResNet, BasicBlock

In [53]:
class ActivatedBatchNormFN(torch.autograd.Function):
    # from: http://cthorey.github.io./backpropagation/

    @staticmethod
    def forward(ctx, x, gamma, beta):

        mu = x.mean(dim=(0, 2, 3), keepdim=True)
        var = ((x - mu) ** 2).mean(dim=(0, 2, 3), keepdim=True)

        xhat = (x - mu) / torch.sqrt(var + 1e-5)

        bn_out = gamma * xhat + beta

        out = bn_out.clamp(min=0)

        ctx.save_for_backward(x, gamma, bn_out)

        return out

    @staticmethod
    def backward(ctx, dout):
        x, gamma, bn_out = ctx.saved_tensors
        dx = dgamma = dbeta = None

        dout = dout * (bn_out > 0)

        N = dout.shape[0] * dout.shape[2] * dout.shape[3]

        dbeta = dout.sum(dim=(0, 2, 3), keepdim=True)

        mu = x.mean(dim=(0, 2, 3), keepdim=True)
        var = ((x - mu) ** 2).mean(dim=(0, 2, 3), keepdim=True)

        dgamma = torch.sum(((x - mu) / torch.sqrt(var + 1e-5))
                           * dout, dim=(0, 2, 3), keepdim=True)

        dx = (1.0 / N) * gamma * (1.0 / torch.sqrt(var + 1e-5)) * (N * dout - torch.sum(dout, dim=(0, 2, 3),
                                                                                        keepdim=True) - ((x - mu) * ((var + 1e-5) ** -1.0) * torch.sum(dout * (x - mu), dim=(0, 2, 3), keepdim=True)))

        return dx, dgamma, dbeta

In [54]:
bn = ActivatedBatchNormFN.apply

In [55]:
x = torch.rand((1, 2, 4, 4), requires_grad=True)
gamma = torch.ones((1, 2, 1, 1), requires_grad=True)
beta = torch.zeros((1, 2, 1, 1), requires_grad=True)

In [56]:
out = bn(x, gamma, beta)

In [57]:
temp = out.sum()
temp.backward()

In [58]:
x.mean(), x.var()

(tensor(0.5021, grad_fn=<MeanBackward0>),
 tensor(0.0805, grad_fn=<VarBackward0>))

In [59]:
out.mean(), out.var()

(tensor(0.4276, grad_fn=<MeanBackward0>),
 tensor(0.3168, grad_fn=<VarBackward0>))

In [61]:
from torch.autograd import gradcheck

x = torch.rand((1, 2, 4, 4), dtype=torch.double, requires_grad=True)
gamma = torch.ones((1, 2, 1, 1), dtype=torch.double, requires_grad=True)
beta = torch.zeros((1, 2, 1, 1), dtype=torch.double, requires_grad=True)

input = (x, gamma, beta)
test = gradcheck(bn, input, eps=1e-6, atol=1e-4)
print(test)

True


In [34]:
mod = ActivatedBatchNormAutograd(2)

NameError: name 'ActivatedBatchNormAutograd' is not defined

In [ ]:
x = torch.rand((1, 2, 4, 4), dtype=torch.double, requires_grad=True)

In [ ]:
out = mod(x)

In [ ]:
out.mean(), out.var()